In [1]:
import pandas as pd

In [2]:
path ="/Users/imane/Downloads/scrapenglish_data.xlsx"
Dataset = pd.read_excel(path)
Dataset

,Unnamed: 0,internal_id,domain,IABCategory,title,description,FullText,language,detected_language
0,0,8643889,touristscavengerhunt.com,/Travel,tourist scavenger hunt self guided walking tou...,explore the city with our 2-3 hour tourist sca...,Tourist Scavenger Hunt self guided walking tou...,en,en
1,2,1265895,plannedspontaneityhiking.com,/Hobbies & Leisure/Outdoors/Hiking & Camping,planned spontaneity â€“ a blog about hiking ar...,a blog about hiking around the midwest and beyond,Planned Spontaneity – A blog about hiking arou...,en,en
2,4,8429739,www.utvdirect.com,/Autos & Vehicles/Motor Vehicles (By Type)/Off...,utv accessories & parts | find the best side b...,find the right side by side accessories for yo...,UTV Accessories & Parts | Find the Best Side b...,en,en
3,5,7030283,www.debrovys.com,/Business & Industrial/Construction & Maintena...,"tarps, large canvas tarp, & canvas tarp manufa...",debrovy's is the ideal tarp manufacturer in th...,Industrial Tarp Company & Manufacturer | DeBro...,en,en
4,7,1359840,www.urbanbarn.com,/Home & Garden/Home Furnishings,urban barn,furniture and accessories for your home and of...,"Urban Barn | Furniture & Décor | Living, Dinin...",en,en
...,...,...,...,...,...,...,...,...,...
69,92,1776564,www.perfectfitblinduk.com,/Home & Garden/Home Furnishings,"perfect fit blinds uk - cellular, roller, plea...","perfect fit window blinds - no drill, no holes...",Perfect Fit Blinds UK - Factory Direct Cellula...,en,en
70,93,1491937,www.grayandsons.com,/Shopping/Apparel/Clothing Accessories,used luxury jewelry and watches | gray & sons ...,with our selection of used jewelry and watches...,Used Luxury Jewelry and Watches | Gray & Sons ...,en,en
71,94,370740,www.pfmplus.co.uk,/Business & Industrial/Hospitality Industry/Fo...,parkers food machinery plus,NaN,Home - PFM PlusSkip to contentT: 0330 058 4650...,en,en
72,95,2687093,girlsplaybaseball.blog,/Sports/Team Sports/Baseball,girls play baseballÂ® | for girls who just / w...,for girls who just / want to play baseball,Girls Play Baseball® | For girls who just / wa...,en,en


In [3]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
 !pip install nltk


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/imane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
!pip install spacy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.5 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [9]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [10]:

df_new = pd.DataFrame(Dataset,columns=['FullText','description','title'])
df_new["title_description"] =  df_new["title"].fillna(" ") + " " + df_new["description"].fillna(" ")
#Concatenate the "title" and "description" columns, filling any missing values with empty strings
df_new.head(50)

,FullText,description,title,title_description
0,Tourist Scavenger Hunt self guided walking tou...,explore the city with our 2-3 hour tourist sca...,tourist scavenger hunt self guided walking tou...,tourist scavenger hunt self guided walking tou...
1,Planned Spontaneity – A blog about hiking arou...,a blog about hiking around the midwest and beyond,planned spontaneity â€“ a blog about hiking ar...,planned spontaneity â€“ a blog about hiking ar...
2,UTV Accessories & Parts | Find the Best Side b...,find the right side by side accessories for yo...,utv accessories & parts | find the best side b...,utv accessories & parts | find the best side b...
3,Industrial Tarp Company & Manufacturer | DeBro...,debrovy's is the ideal tarp manufacturer in th...,"tarps, large canvas tarp, & canvas tarp manufa...","tarps, large canvas tarp, & canvas tarp manufa..."
4,"Urban Barn | Furniture & Décor | Living, Dinin...",furniture and accessories for your home and of...,urban barn,urban barn furniture and accessories for your ...
5,"Fishing Directory Website for Listings, Report...","fishing directory website for fishing tips, fi...","fishing directory website for listings, report...","fishing directory website for listings, report..."
6,Animal Stock Health and Veterinary Products Ir...,NaN,animal stock health and veterinary products ir...,animal stock health and veterinary products ir...
7,"Palm Stones, Worry Stones, Tumbled Stones, Han...","highest quality palm stones, worry stones, tum...","palm stones, worry stones, tumbled stones, han...","palm stones, worry stones, tumbled stones, han..."
8,Islamic Books | DarussalamCurrency:GBPUS Dolla...,darussalam: a global leader in islamic books,darussalam,darussalam darussalam: a global leader in isla...
9,Grey Bruce Public Health HomepageAccessibility...,NaN,\tgrey bruce public health page,\tgrey bruce public health page


In [11]:
df_new_data = pd.DataFrame(df_new,columns=['FullText','title_description'])

df_new_data

,FullText,title_description
0,Tourist Scavenger Hunt self guided walking tou...,tourist scavenger hunt self guided walking tou...
1,Planned Spontaneity – A blog about hiking arou...,planned spontaneity â€“ a blog about hiking ar...
2,UTV Accessories & Parts | Find the Best Side b...,utv accessories & parts | find the best side b...
3,Industrial Tarp Company & Manufacturer | DeBro...,"tarps, large canvas tarp, & canvas tarp manufa..."
4,"Urban Barn | Furniture & Décor | Living, Dinin...",urban barn furniture and accessories for your ...
...,...,...
69,Perfect Fit Blinds UK - Factory Direct Cellula...,"perfect fit blinds uk - cellular, roller, plea..."
70,Used Luxury Jewelry and Watches | Gray & Sons ...,used luxury jewelry and watches | gray & sons ...
71,Home - PFM PlusSkip to contentT: 0330 058 4650...,parkers food machinery plus
72,Girls Play Baseball® | For girls who just / wa...,girls play baseballÂ® | for girls who just / w...


In [12]:
data = [f"{col1} - {col2}" for col1, col2 in zip(df_new_data['FullText'], df_new_data['title_description'])]

data

["Tourist Scavenger Hunt self guided walking tours around the worldSkip to navigationSkip to contentShopMy account / LoginConciergeTravel AgentsContact UsSearch for:SearchMenuLocationsCanadaAlbertaNew BrunswickNova ScotiaOntarioKingston, OnOttawa, OnToronto, OnQuebecGatineau, QcMontreal, QcOld Terrebonne, QcQuebec City, QcSherbrooke, QcTrois-Rivieres, QcEuropeAustriaBelgiumCzech RepublicFranceHungaryNetherlandsPortugalUnited KingdomEnglandBathLeedsLondonManchesterOxfordSouthamptonYorkWalesUnited-StatesBostonCaliforniaNew York CityPhiladelphiaWashington D.C.ShopOn SaleBest SellingTravel Agent accessConcierge PartnershipHow it worksWho can play?Regular or Explorer?Why Choose Us ?Concierge PartnershipTravel Agent accessAbout UsFAQNewsletterBlogContact UsAccountMy CartMy Scavenger Hunt ResultsCookie PolicyEnglishFrançaisEnglishFrançaisShopBy Country & RegionBest SellingOn SaleMy accountMy CartHow it worksCall: 514-949-5201Text: 475-215-1013FAQContact Us$0.000 items75 Adventures in 44 citie

In [13]:
#As we can see there are many  newline and extra spaces that is quite distracting. Let’s get rid of them using regular expressions
# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]
data = [re.sub("&", "", str(sent)) for sent in data]
data = [re.sub("-", "", str(sent)) for sent in data]
data = [re.sub("!", "", str(sent)) for sent in data]
data = [re.sub("/", "", str(sent)) for sent in data]
data = [re.sub(",", "", str(sent)) for sent in data]
data = [re.sub("'", "", str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

print(data[:5])

['Tourist Scavenger Hunt self guided walking tours around the worldSkip to navigationSkip to contentShopMy account LoginConciergeTravel AgentsContact UsSearch for:SearchMenuLocationsCanadaAlbertaNew BrunswickNova ScotiaOntarioKingston OnOttawa OnToronto OnQuebecGatineau QcMontreal QcOld Terrebonne QcQuebec City QcSherbrooke QcTroisRivieres QcEuropeAustriaBelgiumCzech RepublicFranceHungaryNetherlandsPortugalUnited KingdomEnglandBathLeedsLondonManchesterOxfordSouthamptonYorkWalesUnitedStatesBostonCaliforniaNew York CityPhiladelphiaWashington D.C.ShopOn SaleBest SellingTravel Agent accessConcierge PartnershipHow it worksWho can play?Regular or Explorer?Why Choose Us ?Concierge PartnershipTravel Agent accessAbout UsFAQNewsletterBlogContact UsAccountMy CartMy Scavenger Hunt ResultsCookie PolicyEnglishFrançaisEnglishFrançaisShopBy Country RegionBest SellingOn SaleMy accountMy CartHow it worksCall: 5149495201Text: 4752151013FAQContact Us$0.000 items75 Adventures in 44 cities across North Amer

In [14]:
#tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
#gensim.utils.simple_preprocess : Convert a document into a list of tokens.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:20])

[['tourist', 'scavenger', 'hunt', 'self', 'guided', 'walking', 'tours', 'around', 'the', 'worldskip', 'to', 'navigationskip', 'to', 'contentshopmy', 'account', 'agentscontact', 'ussearch', 'for', 'brunswicknova', 'onottawa', 'ontoronto', 'qcmontreal', 'qcold', 'terrebonne', 'qcquebec', 'city', 'qcsherbrooke', 'qctroisrivieres', 'york', 'shopon', 'salebest', 'sellingtravel', 'agent', 'accessconcierge', 'partnershiphow', 'it', 'workswho', 'can', 'play', 'regular', 'or', 'explorer', 'why', 'choose', 'us', 'concierge', 'agent', 'accessabout', 'usaccountmy', 'cartmy', 'scavenger', 'hunt', 'resultscookie', 'country', 'regionbest', 'sellingon', 'salemy', 'accountmy', 'carthow', 'it', 'workscall', 'text', 'faqcontact', 'us', 'items', 'adventures', 'in', 'cities', 'across', 'north', 'america', 'and', 'europe', 'tourist', 'scavenger', 'hunt', 'is', 'hour', 'selfguided', 'walking', 'tour', 'with', 'challenges', 'to', 'solve', 'as', 'you', 'learn', 'some', 'history', 'see', 'the', 'sites', 'and', 

In [15]:
import gensim
from gensim.models import Phrases

# we have a list of tokenized documents named 'data_words'
# Each document in 'data_words' should be a list of tokens

# Build the bigram model
bigram = Phrases(data_words, min_count=5, threshold=100) 
bigram_model = gensim.models.phrases.Phraser(bigram)

# Apply the bigram model to the data words
data_words_bigram = [bigram_model[doc] for doc in data_words]  

# Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more
# Build the trigram model
trigram = Phrases(data_words_bigram, min_count=5, threshold=100) 
trigram_model = gensim.models.phrases.Phraser(trigram)

# Apply the trigram model to the data words
data_words_trigram = [trigram_model[bigram_model[doc]] for doc in data_words] 
# Example usage:
# Accessing the data_words with bigrams and trigrams
for doc in data_words_trigram:
    print(doc[:20])

['tourist_scavenger_hunt', 'self', 'guided', 'walking', 'tours', 'around', 'the', 'worldskip', 'to', 'navigationskip', 'to', 'contentshopmy', 'account', 'agentscontact', 'ussearch', 'for', 'brunswicknova', 'onottawa', 'ontoronto', 'qcmontreal']
['planned', 'spontaneity', 'blog', 'about', 'hiking', 'around', 'the', 'midwest', 'and', 'beyondabout', 'goalplanned', 'spontaneitya', 'blog', 'about', 'hiking', 'around', 'the', 'midwest', 'and', 'beyondcache']
['utv', 'accessories', 'parts', 'find', 'the', 'best', 'side', 'by', 'side', 'accessories', 'parts', 'at', 'the', 'best', 'prices', 'utv', 'directwishlist', 'my', 'accountsign', 'inorcreate']
['industrial', 'tarp', 'company', 'manufacturer', 'debrovy', 'squestions', 'feedback', 'powered', 'byolark', 'live', 'chat', 'softwaretoggle', 'menusearchsign', 'inorcreate', 'an', 'productsall', 'aluminum', 'productsmerritt', 'aluminum', 'products']
['urban_barn', 'furniture', 'decor', 'living', 'dining', 'bedroom', 'urban_barn', 'francais', 'engli

In [16]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_model[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
### Do lemmatization keeping only noun, adj, vb, adv

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigram = [trigram_model[bigram_model[doc]] for doc in data_words]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


print(data_lemmatized[:1])

[['hunt', 'self', 'guide', 'walking', 'tour', 'navigationskip', 'contentshopmy', 'account', 'city', 'saleb', 'sellingtravel', 'agent', 'accessconcierge', 'partnershiphow', 'play', 'regular', 'explorer', 'choose', 'concierge', 'agent', 'accessabout', 'workscall', 'text', 'faqcontact', 'item', 'adventure', 'city', 'hour', 'selfguided_walke', 'tour', 'challenge', 'solve', 'learn', 'history', 'see', 'site', 'historic', 'center', 'add', 'add', 'add', 'cartnew', 'brunswick', 'scotiahalifax', 'walking_tourrate', 'add', 'cartsydney', 'n', 'walking_tourrate', 'add', 'add', 'add', 'n', 'add', 'add', 'add', 'add', 'add', 'mini', 'add', 'add', 'civic', 'center', 'add', 'walking_tourrate', 'add', 'add', 'embassy', 'add', 'cartqueen', 'park', 'add', 'olympic', 'park', 'add', 'cartgatineau', 'add', 'plateau', 'mountroyal', 'add', 'cartquebec', 'city', 'upper', 'town', 'add', 'cartmontreal', 'entertainment', 'district', 'add', 'cartmontreal', 'downtown', 'add', 'cartlachine', 'selfguided_walke', 'add'

In [18]:
##Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
##For example, (0, 1) above implies word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])



[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 70), (9, 2), (10, 2), (11, 2), (12, 3), (13, 2), (14, 8), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 2), (23, 2), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 2), (52, 3), (53, 1), (54, 1), (55, 2), (56, 2), (57, 1), (58, 7), (59, 1), (60, 1), (61, 13), (62, 1), (63, 1), (64, 4), (65, 2), (66, 4), (67, 1), (68, 2), (69, 1), (70, 1), (71, 20), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 4), (80, 1), (81, 1), (82, 1), (83, 3), (84, 1), (85, 2), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 4), (94, 1), (95, 2), (96, 1), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 4), (106, 1), (107, 1), (108, 1), (109, 3), (110,

In [19]:
id2word[0]

'absolutely'

In [20]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolutely', 1),
  ('accept', 1),
  ('acceptpowere', 1),
  ('accessabout', 1),
  ('accessconcierge', 1),
  ('account', 1),
  ('activity', 1),
  ('ad', 2),
  ('add', 70),
  ('adventure', 2),
  ('agent', 2),
  ('also', 2),
  ('analyze', 3),
  ('autosave', 2),
  ('available', 8),
  ('availablecity', 1),
  ('basic', 2),
  ('belem', 1),
  ('betatest', 2),
  ('bounce', 1),
  ('brainchild', 2),
  ('browse', 1),
  ('browser', 2),
  ('brunswick', 2),
  ('buildingplan', 1),
  ('button', 2),
  ('buy', 2),
  ('campaign', 1),
  ('canadian', 1),
  ('cartbrooklyn', 1),
  ('cartchelsea', 1),
  ('cartczech', 1),
  ('cartdupont', 1),
  ('cartfisherman', 1),
  ('cartgatineau', 1),
  ('cartharlem', 1),
  ('carthollywood', 1),
  ('cartlachine', 1),
  ('cartlisbon', 2),
  ('cartlondon', 1),
  ('cartmaastricht', 1),
  ('cartmontreal', 2),
  ('cartnew', 1),
  ('cartprague', 1),
  ('cartquebec', 1),
  ('cartqueen', 1),
  ('cartrotterdam', 1),
  ('cartsherbrooke', 1),
  ('cartsydney', 1),
  ('cartteam', 1),

In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
# The above LDA model is built with 15 different topics where each topic is a combination of keywords, 
#and each keyword contributes a certain weightage to the topic.
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"add" + 0.009*"sale" + 0.007*"tool" + 0.006*"accessory" + '
  '0.006*"school" + 0.005*"good" + 0.005*"select" + 0.005*"power" + '
  '0.005*"condo" + 0.004*"cookie"'),
 (1,
  '0.012*"read" + 0.009*"likeloade" + 0.008*"tour" + 0.007*"day" + '
  '0.006*"travel" + 0.006*"luxury" + 0.006*"trek" + 0.005*"watch" + '
  '0.005*"translation" + 0.005*"usd_rate"'),
 (2,
  '0.017*"quick" + 0.010*"year" + 0.009*"power" + 0.008*"add" + 0.008*"day" + '
  '0.006*"see" + 0.005*"week" + 0.005*"product" + 0.005*"photo" + '
  '0.004*"basketquick"'),
 (3,
  '0.024*"original_price" + 0.024*"price_original" + 0.023*"price" + '
  '0.013*"price_current" + 0.011*"current_price" + 0.010*"add" + '
  '0.009*"perfect_fit" + 0.006*"gbp" + 0.006*"view_full" + 0.005*"day"'),
 (4,
  '0.023*"price" + 0.012*"regular" + 0.011*"door" + 0.006*"add" + '
  '0.004*"material" + 0.004*"metal" + 0.004*"roach" + 0.003*"part" + '
  '0.003*"line" + 0.003*"pike"'),
 (5,
  '0.011*"online_form" + 0.009*"woman" + 0.009*"man

In [23]:
!pip install pyLDAvis --user


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [24]:
!pip install pyLDAvis


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [25]:
!pip install pyLDAvis gensim


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [26]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [27]:
!pip install "pandas>=2.0.0"


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [28]:
!pip install --upgrade pandas


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [29]:
!pip install gensim


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.090487  0.139073       1        1  21.855632
12    -0.084105  0.127663       2        1  12.236027
8     -0.246275 -0.224878       3        1   8.995167
3      0.177909 -0.126876       4        1   8.896619
2      0.024289  0.052991       5        1   7.229789
5      0.058533 -0.007384       6        1   6.811390
0      0.038868  0.009720       7        1   5.875364
11    -0.038782  0.061125       8        1   5.541419
1     -0.041455  0.050769       9        1   4.582750
6     -0.003238 -0.029857      10        1   4.112295
10     0.001488 -0.024841      11        1   3.681749
13    -0.007602  0.051406      12        1   3.584388
9      0.090340 -0.025734      13        1   2.365873
14     0.048294 -0.010478      14        1   2.298225
4      0.072223 -0.042698      15        1   1.933312, topic_info=            Term        Freq       Total Category  logprob  loglift
64         click  388.000000  388.000000  Default  30.0000  30.0000
4403  new_window  337.000000  337.000000  Default  29.0000  29.0000
216        share  372.000000  372.000000  Default  28.0000  28.0000
8            add  411.000000  411.000000  Default  27.0000  27.0000
473        price  180.000000  180.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
208          see    2.727723  105.940481  Topic15  -6.0434   0.2865
767         shop    2.218805   31.584846  Topic15  -6.2499   1.2902
712      product    2.210672   98.954658  Topic15  -6.2535   0.1446
552         back    2.175170   53.526943  Topic15  -6.2697   0.7429
1422      review    2.171866   25.484514  Topic15  -6.2712   1.4835

[839 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
538        1  0.049854            _
538        4  0.008309            _
538        7  0.074780            _
538        8  0.830894            _
538        9  0.008309            _
...      ...       ...          ...
6278       1  0.873869  york_yankee
10074     10  0.970268  youtube_com
13573      1  0.920273           zd
3091      13  0.789014           কভ
3092      13  0.789215         করআন

[2144 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 13, 9, 4, 3, 6, 1, 12, 2, 7, 11, 14, 10, 15, 5])

In [31]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.137720605539288

Coherence Score:  0.38247685318551927


In [ ]:
 #the perplexity value is -8.137720605539288
#Perplexity is a negative value, and lower values indicate better performance. 
#Since the perplexity is negative, it suggests that the model is reasonably good at predicting the given corpus.
#The coherence score ranges from 0 to 1, where higher values indicate better-defined and more interpretable topics. 
#A coherence score of 0.3824 suggests that the topics generated by the model have some level of coherence but might still need improvement.